In [ ]:
from src.material.material.phong_material import *
from src import World
from src import Sphere
from src import PointLight
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src.render.loops import *
from src import Plane
from src.render.loops.progress import ProgressDisplay, PreviewConfig
from src.render.render_config import RenderConfig
import cProfile, pstats
from src import ipynb_display_images

In [ ]:
gold = PhongMaterial(
    name="gold",
    base_color=Color.custom_rgb(255, 215, 0),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=100,
    ior=0.47,
    reflectivity=0.9,
    transparency=0.0
)

glossy_green = PhongMaterial(
    name="glossy_green",
    base_color=Color.custom_rgb(0, 255, 0),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=50,
    ior=1.5,
    reflectivity=0.4,
    transparency=0.0
)

sphere = Sphere(
    center=Vertex(0, 0.2, -5),
    radius=1.0,
    material=gold
)

plane = Plane(
    point=Vertex(0, -1, 0),
    normal=Vector(0, 1, 0),
    material=glossy_green
)


world = World()
world.add(sphere, plane)

point_light = PointLight(
    position=Vertex(5, 5, 0),
    intensity=2000.0,
    falloff=0.01,
)

camera = Camera(
    fov = 40,
    aspect_ratio = Resolution.R360p.aspect_ratio,
    origin = Vertex(0, 0, 0),
    direction = Vector(0, 0, -1),
)

scene = Scene(
    camera = camera,
    world = world,
    lights = [point_light],
    skybox_path = None,
)

scene.validate()

In [ ]:
render_config = RenderConfig(
    resolution=Resolution.R360p,
    samples_per_pixel=1,
    max_depth=4,
)

preview_config = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

multi_process_ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
)

ray_tracer = LinearRayCaster(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
)


In [ ]:
png = multi_process_ray_tracer.render("images/my_first_multi_process_render.png")
ipynb_display_images(png)

In [ ]:
png = ray_tracer.render("images/my_first_multi_process_render.png")
ipynb_display_images(png)

In [ ]:
def run():
    ray_tracer.render("images/single_core.png")

cProfile.run("run()", "benchmark_data/profile.out")
stats = pstats.Stats("benchmark_data/profile.out")
stats.sort_stats("cumtime").print_stats(100)

In [ ]:
def run_multiprocess():
    multi_process_ray_tracer.render("images/multiprocess.png")

cProfile.run("run_multiprocess()", "benchmark_data/profile_multiprocess.out")
stats = pstats.Stats("benchmark_data/profile_multiprocess.out")
stats.sort_stats("cumtime").print_stats(100)